In [8]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import os
import numpy as np
import tensorflow as tf
import pandas as pd

## Loading Training/Testing/Validation Data

In [2]:
dataset = "ChlorineConcentration"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_test.npz')))

x_train = np.reshape(x_train_load['arr_0'], [x_train_load['arr_0'].shape[0], x_train_load['arr_0'].shape[1], 1])
x_test = np.reshape(x_test_load['arr_0'], [x_test_load['arr_0'].shape[0], x_test_load['arr_0'].shape[1], 1])

x_all = np.concatenate((x_train, x_test), axis = 0)

n_instances = x_all.shape[0]

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))

y_train = y_train_load['arr_0']
y_test = y_test_load['arr_0']

y_all = np.concatenate((y_train, y_test), axis = 0)

x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size = 0.3, random_state = 42)

x_valid = x_test[:int(0.1*n_instances), :, :]
y_valid = y_test[:int(0.1*n_instances)]

x_test = x_test[int(0.1*n_instances):, :, :]
y_test = y_test[int(0.1*n_instances):]

print(f"x_train: {x_train.shape} - y_train: {y_train.shape}")
print(f"x_test: {x_test.shape} - y_test: {y_test.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

x_train: (3014, 166, 1) - y_train: (3014,)
x_test: (863, 166, 1) - y_test: (863,)
x_valid: (430, 166, 1) - y_valid: (430,)


In [3]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

model = Sequential()

model.add(RNN(NeuromodulatedBistableRecurrentCellLayer(output_dim = 212),
              input_shape = (None, 1), 
              return_sequences = False))
model.add(Dense(to_categorical(y_train).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 212)               90948     
_________________________________________________________________
dense (Dense)                (None, 4)                 852       
Total params: 91,800
Trainable params: 91,800
Non-trainable params: 0
_________________________________________________________________


In [5]:
n_fold = 1

for train, test in sdk.split(x_all, y_all):

    file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-nBRC_{n_fold}.hdf5'))

    callbacks = [
        ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
        EarlyStopping(monitor = 'val_loss', patience = 20, mode = 'min')]

    result = model.fit(x_all[train], 
                       to_categorical(y_all[train]), 
                       epochs = 2000, 
                       batch_size = 100, 
                       validation_data = (x_all[test], to_categorical(y_valid)), 
                       callbacks = callbacks)

    df_results = pd.DataFrame(result.history)
    df_results.to_csv(os.path.abspath(os.path.join('models', dataset, 'nBRC_results.csv')))

Train on 3014 samples, validate on 430 samples
Epoch 1/2000
3000/3014 [============================>.] - ETA: 0s - loss: 0.9939 - acc: 0.5560
Epoch 00001: val_loss improved from inf to 0.96564, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\ChlorineConcentration\ChlorineConcentration-nBRC.hdf5
3014/3014 [==============================] - 3s 1ms/sample - loss: 0.9932 - acc: 0.5571 - val_loss: 0.9656 - val_acc: 0.5977
Epoch 2/2000
3000/3014 [============================>.] - ETA: 0s - loss: 0.9908 - acc: 0.5597
Epoch 00002: val_loss improved from 0.96564 to 0.96376, saving model to C:\Users\Sergio\Documents\GitHub_Repositories\01_Maintained\NeuralODE\models\ChlorineConcentration\ChlorineConcentration-nBRC.hdf5
3014/3014 [==============================] - 3s 1ms/sample - loss: 0.9910 - acc: 0.5594 - val_loss: 0.9638 - val_acc: 0.5977
Epoch 3/2000
3000/3014 [============================>.] - ETA: 0s - loss: 1.0033 - acc: 0.5423
Epoch 00003: val

In [7]:
saved_epoch = np.argmin(result.history['val_loss'])

print(f"Epoch {saved_epoch}: {result.history['val_acc'][saved_epoch]} val_acc")

model.load_weights(file_path)
model.evaluate(x_test, to_categorical(y_test))

Epoch 51: 0.5976744294166565 val_acc
863/863 [==============================] - 1s 868us/sample - loss: 0.9945 - acc: 0.5458


[0.9945175275465566, 0.5457706]